In [1]:
import pandas as pd
import numpy as np
import torch

In [5]:
csvpath = 'test_csv.csv'
win_size = 10
win_shift = 5

df = pd.read_csv(csvpath, sep=" ")
df

,accx,accy,accz,gyrox,gyroy,gyroz,labels
0,0.459151,-0.591990,-0.530524,0.167089,0.026853,0.077674,5
1,0.479760,-0.581099,-0.545450,0.152189,0.059052,0.172579,5
2,0.503805,-0.591990,-0.565352,0.169145,0.080519,0.185095,5
3,0.541590,-0.595621,-0.411115,0.146537,0.276398,0.205953,5
4,0.459151,-0.591990,-0.316582,0.265226,0.394462,0.211168,5
...,...,...,...,...,...,...,...
175297,0.637307,-0.557568,-0.814022,-0.695706,-0.760739,0.406168,2
175298,0.765722,-0.513891,-0.810846,-0.852813,-0.660414,0.453271,2
175299,0.620410,-0.867672,-1.236365,-0.272249,-1.654690,0.630746,2
175300,1.377383,-1.618912,-1.334806,-0.446249,0.255559,0.667755,2


In [7]:
list_feat = []
for i in range(0, int((len(df) - win_size) / win_shift) + 1):
    list_feat.append(
        df[i * win_shift : i * win_shift + win_size]
    )

list_feat[:3]

[       accx      accy      accz     gyrox     gyroy     gyroz  labels
 0  0.459151 -0.591990 -0.530524  0.167089  0.026853  0.077674       5
 1  0.479760 -0.581099 -0.545450  0.152189  0.059052  0.172579       5
 2  0.503805 -0.591990 -0.565352  0.169145  0.080519  0.185095       5
 3  0.541590 -0.595621 -0.411115  0.146537  0.276398  0.205953       5
 4  0.459151 -0.591990 -0.316582  0.265226  0.394462  0.211168       5
 5  0.366406 -0.533902 -0.187223  0.309927  0.386412  0.105832       5
 6  0.345796 -0.508488 -0.142444  0.188669  0.350635 -0.077722       5
 7  0.431671 -0.515749 -0.092690 -0.074912  0.359580 -0.186186       5
 8  0.541590 -0.541163 -0.107616 -0.149927  0.460650 -0.104838       5
 9  0.514110 -0.577468 -0.212099 -0.154552  0.504477 -0.143426       5,
         accx      accy      accz     gyrox     gyroy     gyroz  labels
 5   0.366406 -0.533902 -0.187223  0.309927  0.386412  0.105832       5
 6   0.345796 -0.508488 -0.142444  0.188669  0.350635 -0.077722       5
 7

In [30]:
y_arr = np.array(list_feat, dtype=np.int32)[:, :, 6]
np.bincount(y_arr[729])

array([0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 7])

In [10]:
y_lst = []
for e in range(len(y_arr)):
    cn = np.bincount(y_arr[e])
    y_lst.append(np.argmax(cn))
    print(cn)
    print(y_lst)
    break

[ 0  0  0  0  0 10]
[5]


In [32]:
X = torch.tensor(np.array(list_feat)[:, :, :6], dtype=torch.float)
y = torch.tensor(np.array(y_lst) - 1, dtype=torch.long)
X[0]

tensor([[ 0.4592, -0.5920, -0.5305,  0.1671,  0.0269,  0.0777],
        [ 0.4798, -0.5811, -0.5455,  0.1522,  0.0591,  0.1726],
        [ 0.5038, -0.5920, -0.5654,  0.1691,  0.0805,  0.1851],
        [ 0.5416, -0.5956, -0.4111,  0.1465,  0.2764,  0.2060],
        [ 0.4592, -0.5920, -0.3166,  0.2652,  0.3945,  0.2112],
        [ 0.3664, -0.5339, -0.1872,  0.3099,  0.3864,  0.1058],
        [ 0.3458, -0.5085, -0.1424,  0.1887,  0.3506, -0.0777],
        [ 0.4317, -0.5157, -0.0927, -0.0749,  0.3596, -0.1862],
        [ 0.5416, -0.5412, -0.1076, -0.1499,  0.4606, -0.1048],
        [ 0.5141, -0.5775, -0.2121, -0.1546,  0.5045, -0.1434]])

In [37]:
num_node = 6
adjacency = np.zeros((num_node, num_node))

def normalize_digraph(A):
    Dl = np.sum(A, 0)
    num_node = A.shape[0]
    Dn = np.zeros((num_node, num_node))
    for i in range(num_node):
        if Dl[i] > 0:
            Dn[i, i] = Dl[i]**(-1)
    AD = np.dot(A, Dn)
    return AD

normalize_adjacency = normalize_digraph(adjacency)
A = np.zeros((1, num_node, num_node))
A[0] = normalize_adjacency
A

array([[[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]]])

In [40]:
import numpy as np

# Define example data
batch_size = 2
num_nodes = 3
num_features = 2  # x and y positions
num_timesteps = 5

# Create example input data
input_data = np.array([
    # Sequence 1
    [
        # Time step 1
        [
            [0.1, 0.2],  # Head (x, y)
            [0.3, 0.4],  # Left hand (x, y)
            [0.5, 0.6]   # Right hand (x, y)
        ],
        # Time step 2
        [
            [0.2, 0.3],
            [0.4, 0.5],
            [0.6, 0.7]
        ],
        # Time step 3
        [
            [0.3, 0.4],
            [0.5, 0.6],
            [0.7, 0.8]
        ],
        # Time step 4
        [
            [0.4, 0.5],
            [0.6, 0.7],
            [0.8, 0.9]
        ],
        # Time step 5
        [
            [0.5, 0.6],
            [0.7, 0.8],
            [0.9, 1.0]
        ],
    ],
    # Sequence 2
    [
        # Time step 1
        [
            [0.2, 0.3],
            [0.4, 0.5],
            [0.6, 0.7]
        ],
        # Time step 2
        [
            [0.3, 0.4],
            [0.5, 0.6],
            [0.7, 0.8]
        ],
        # Time step 3
        [
            [0.4, 0.5],
            [0.6, 0.7],
            [0.8, 0.9]
        ],
        # Time step 4
        [
            [0.5, 0.6],
            [0.7, 0.8],
            [0.9, 1.0]
        ],
        # Time step 5
        [
            [0.6, 0.7],
            [0.8, 0.9],
            [1.0, 1.1]
        ],
    ]
])

print(input_data.shape)
# Reshape to match (batch_size, num_nodes, num_features, num_timesteps)
input_data = input_data.transpose(0, 2, 3, 1)

print(input_data.shape)  # (2, 3, 5, 2)
print(input_data)


(2, 5, 3, 2)
(2, 3, 2, 5)
[[[[0.1 0.2 0.3 0.4 0.5]
   [0.2 0.3 0.4 0.5 0.6]]

  [[0.3 0.4 0.5 0.6 0.7]
   [0.4 0.5 0.6 0.7 0.8]]

  [[0.5 0.6 0.7 0.8 0.9]
   [0.6 0.7 0.8 0.9 1. ]]]


 [[[0.2 0.3 0.4 0.5 0.6]
   [0.3 0.4 0.5 0.6 0.7]]

  [[0.4 0.5 0.6 0.7 0.8]
   [0.5 0.6 0.7 0.8 0.9]]

  [[0.6 0.7 0.8 0.9 1. ]
   [0.7 0.8 0.9 1.  1.1]]]]


In [1]:
import pandas as pd
import numpy as np
import torch
from glob import glob


In [2]:
csv_dir_path = "Data/csv-noise-1"
csv_files = glob(csv_dir_path+'/*.csv')
csv_files.sort()
len(csv_files)

113

In [3]:
datacsv = pd.read_csv(csv_files[0]).values
mean =  np.mean(datacsv)
std_dev = np.std(datacsv)
print(mean, std_dev)

3.6940022493245626 20.568360381572464


In [4]:
win_size_in = 25
win_size_out = 10
win_shift = 1
win_size_total = win_size_in + win_size_out

In [26]:
filedata = torch.tensor(pd.read_csv(csv_files[0]).values, dtype=torch.float)
filedata[0:3, 3:-2].view(-1, 3, 9).permute(0, 2, 1)
# newdata.view(-1, 3, 9).permute(0, 2, 1)
new = filedata[0:3, 3:-2].view(-1, 3, 9).permute(0, 2, 1)
new

tensor([[[ 2.0233e-04, -3.1873e-03, -7.6071e-02],
         [-7.8527e-01,  1.3901e-02,  3.9760e+00],
         [-2.5058e-04, -9.4234e-03,  1.2604e+00],
         [-2.3557e+00,  3.9718e-03, -2.1618e+01],
         [-4.5420e-05, -8.7396e-03, -5.0819e-01],
         [ 1.5720e+00,  1.5750e-02, -2.3370e+00],
         [ 7.8497e-01, -7.2307e-03, -1.0937e-02],
         [ 3.4927e-02, -4.3984e-04, -9.4647e-04],
         [ 3.4927e-02, -4.3984e-04, -9.4647e-04]],

        [[ 1.8075e-04, -6.0197e-03, -5.3287e-03],
         [-7.8472e-01,  2.9271e-02,  3.3494e+00],
         [-6.1821e-04,  7.3245e-04,  8.3170e-01],
         [-2.3553e+00,  2.6097e-02, -2.2308e+01],
         [-9.4782e-05,  9.2247e-03, -6.0922e-01],
         [ 1.5724e+00,  9.2008e-03, -2.1702e+00],
         [ 7.8448e-01, -1.9972e-02,  7.9367e-03],
         [ 3.4927e-02, -5.8003e-04, -1.0303e-03],
         [ 3.4927e-02, -5.8003e-04, -1.0303e-03]],

        [[ 1.7817e-04, -6.5480e-03,  2.5495e-01],
         [-7.8383e-01,  2.1782e-02,  3.6409e+0

In [27]:
new.reshape(3, -1)

tensor([[ 2.0233e-04, -3.1873e-03, -7.6071e-02, -7.8527e-01,  1.3901e-02,
          3.9760e+00, -2.5058e-04, -9.4234e-03,  1.2604e+00, -2.3557e+00,
          3.9718e-03, -2.1618e+01, -4.5420e-05, -8.7396e-03, -5.0819e-01,
          1.5720e+00,  1.5750e-02, -2.3370e+00,  7.8497e-01, -7.2307e-03,
         -1.0937e-02,  3.4927e-02, -4.3984e-04, -9.4647e-04,  3.4927e-02,
         -4.3984e-04, -9.4647e-04],
        [ 1.8075e-04, -6.0197e-03, -5.3287e-03, -7.8472e-01,  2.9271e-02,
          3.3494e+00, -6.1821e-04,  7.3245e-04,  8.3170e-01, -2.3553e+00,
          2.6097e-02, -2.2308e+01, -9.4782e-05,  9.2247e-03, -6.0922e-01,
          1.5724e+00,  9.2008e-03, -2.1702e+00,  7.8448e-01, -1.9972e-02,
          7.9367e-03,  3.4927e-02, -5.8003e-04, -1.0303e-03,  3.4927e-02,
         -5.8003e-04, -1.0303e-03],
        [ 1.7817e-04, -6.5480e-03,  2.5495e-01, -7.8383e-01,  2.1782e-02,
          3.6409e+00, -1.2346e-03, -2.2022e-02,  8.4390e-01, -2.3547e+00,
          7.4851e-03, -2.1652e+01, -1.98

In [97]:
filedata = torch.tensor(pd.read_csv(csv_files[0]).values, dtype=torch.float)
joint_feats = filedata[:3, 3:-2].view(-1, 3, 9).permute(0, 2, 1)
otherfeats = filedata[:3, 1:3]
# repeat only first column 3 times
# filedata[:, 1:3].repeat(3, 1)
otherfeats.repeat(9, 1).view(-1, 9, 2)

# concatenate along last dimension
node_feats = torch.cat((joint_feats, otherfeats.repeat(9, 1).view(-1, 9, 2)), dim=-1)
node_feats

tensor([[[ 2.0233e-04, -3.1873e-03, -7.6071e-02,  1.0000e+00,  0.0000e+00],
         [-7.8527e-01,  1.3901e-02,  3.9760e+00,  1.0000e+00,  0.0000e+00],
         [-2.5058e-04, -9.4234e-03,  1.2604e+00,  1.0000e+00,  0.0000e+00],
         [-2.3557e+00,  3.9718e-03, -2.1618e+01,  1.0000e+00,  0.0000e+00],
         [-4.5420e-05, -8.7396e-03, -5.0819e-01,  1.0000e+00,  0.0000e+00],
         [ 1.5720e+00,  1.5750e-02, -2.3370e+00,  1.0000e+00,  0.0000e+00],
         [ 7.8497e-01, -7.2307e-03, -1.0937e-02,  1.0000e+00,  0.0000e+00],
         [ 3.4927e-02, -4.3984e-04, -9.4647e-04,  1.0000e+00,  0.0000e+00],
         [ 3.4927e-02, -4.3984e-04, -9.4647e-04,  1.0000e+00,  0.0000e+00]],

        [[ 1.8075e-04, -6.0197e-03, -5.3287e-03,  1.0000e+00,  0.0000e+00],
         [-7.8472e-01,  2.9271e-02,  3.3494e+00,  1.0000e+00,  0.0000e+00],
         [-6.1821e-04,  7.3245e-04,  8.3170e-01,  1.0000e+00,  0.0000e+00],
         [-2.3553e+00,  2.6097e-02, -2.2308e+01,  1.0000e+00,  0.0000e+00],
         [

In [98]:
# convert from (T, N, F) to (N, T*F)
newnode = node_feats.permute(1, 0, 2)
newnode = newnode.reshape(1, 9, -1).squeeze(0)

In [99]:
# convert from (N, T*F) to (T, N, F)
newnode = newnode.reshape(9, 3, -1).movedim(1, 0)

In [100]:
# convert from (T, N, F) to (T, N*F)
newnode.reshape(3, -1)

tensor([[ 2.0233e-04, -3.1873e-03, -7.6071e-02,  1.0000e+00,  0.0000e+00,
         -7.8527e-01,  1.3901e-02,  3.9760e+00,  1.0000e+00,  0.0000e+00,
         -2.5058e-04, -9.4234e-03,  1.2604e+00,  1.0000e+00,  0.0000e+00,
         -2.3557e+00,  3.9718e-03, -2.1618e+01,  1.0000e+00,  0.0000e+00,
         -4.5420e-05, -8.7396e-03, -5.0819e-01,  1.0000e+00,  0.0000e+00,
          1.5720e+00,  1.5750e-02, -2.3370e+00,  1.0000e+00,  0.0000e+00,
          7.8497e-01, -7.2307e-03, -1.0937e-02,  1.0000e+00,  0.0000e+00,
          3.4927e-02, -4.3984e-04, -9.4647e-04,  1.0000e+00,  0.0000e+00,
          3.4927e-02, -4.3984e-04, -9.4647e-04,  1.0000e+00,  0.0000e+00],
        [ 1.8075e-04, -6.0197e-03, -5.3287e-03,  1.0000e+00,  0.0000e+00,
         -7.8472e-01,  2.9271e-02,  3.3494e+00,  1.0000e+00,  0.0000e+00,
         -6.1821e-04,  7.3245e-04,  8.3170e-01,  1.0000e+00,  0.0000e+00,
         -2.3553e+00,  2.6097e-02, -2.2308e+01,  1.0000e+00,  0.0000e+00,
         -9.4782e-05,  9.2247e-03, -6

In [24]:
for i in range((filedata.shape[0]-win_size_total)//win_shift + 1):
    joint_feats = filedata[i * win_shift : i * win_shift + win_size_in, 3:-2]  # T x N*F
    print(joint_feats[0])
    # convert to T x N x F
    joint_feats = joint_feats.view(win_size_in, -1, 3)
    # transpose only the last two dimensions
    joint_feats = joint_feats.transpose(1, 2)
    print(joint_feats[0])
    break

tensor([ 2.0233e-04, -7.8527e-01, -2.5058e-04, -2.3557e+00, -4.5420e-05,
         1.5720e+00,  7.8497e-01,  3.4927e-02,  3.4927e-02, -3.1873e-03,
         1.3901e-02, -9.4234e-03,  3.9718e-03, -8.7396e-03,  1.5750e-02,
        -7.2307e-03, -4.3984e-04, -4.3984e-04, -7.6071e-02,  3.9760e+00,
         1.2604e+00, -2.1618e+01, -5.0819e-01, -2.3370e+00, -1.0937e-02,
        -9.4647e-04, -9.4647e-04])
torch.Size([3, 9])


In [89]:
data = pd.read_csv('PeMSD7_V_228.csv', header=None).values


In [90]:
mean =  np.mean(data)
std_dev = np.std(data)
print(mean, std_dev)
data.shape

58.889183986244056 13.483268659131536


(12672, 228)

In [91]:
def z_score(x, mean, std):
    """
    Z-score normalization function: $z = (X - \mu) / \sigma $,
    where z is the z-score, X is the value of the element,
    $\mu$ is the population mean, and $\sigma$ is the standard deviation.
    :param x: torch array, input array to be normalized.
    :param mean: float, the value of mean.
    :param std: float, the value of standard deviation.
    :return: torch array, z-score normalized array.
    """
    return (x - mean) / std


data = z_score(data, np.mean(data), np.std(data))

In [92]:
n_window = 21 # 9 pred + 12 hist

for i in range(44):
    for j in range(288 - n_window + 1):
        sta = i * 288 + j
        end = sta + n_window
        new_data = data[sta:end, :]
        full_window = np.swapaxes(data[sta:end, :], 0, 1)
        print(full_window.shape)
        break
    break
        # print(sta, end)

(228, 21)


In [43]:
import torch.nn as nn

criteria = nn.CrossEntropyLoss()

In [45]:
# randome interger between 0 and 6 of size 5
input = torch.randn(5, 6)
target = torch.empty(5, dtype=torch.long).random_(1)
input, target

(tensor([[-0.2994,  0.0541,  1.4420,  0.6157, -0.4392,  0.0760],
         [ 1.6664, -1.2427, -0.1720,  0.4059,  1.1733,  0.6617],
         [ 0.4278, -2.1952,  0.0398, -2.2313, -1.0450, -1.4654],
         [ 0.9222,  1.4038,  0.4078, -0.4102, -0.3342,  0.1266],
         [ 0.4134,  1.2754,  0.7304, -0.1464,  0.6290, -0.4897]]),
 tensor([0, 0, 0, 0, 0]))

In [84]:
target = torch.tensor([1])

tensor([1])

In [100]:
target = torch.randint(5, (3,), dtype=torch.int64)
target

tensor([0, 3, 3])

In [46]:
criteria(input, target)

tensor(1.5270)

In [82]:
m = nn.LogSoftmax(dim=1)
input = torch.randn(2, 3)
output = m(input)
input, output

(tensor([[ 1.1016, -0.6628, -0.1334],
         [ 0.3342,  1.4320,  1.5624]]),
 tensor([[-0.3799, -2.1443, -1.6149],
         [-2.0032, -0.9054, -0.7750]]))

In [105]:
rnn = nn.LSTM(10, 20, 2)
input = torch.randn(5, 3, 10)
h0 = torch.randn(2, 3, 20)
c0 = torch.randn(2, 3, 20)
output, (hn, cn) = rnn(input, (h0, c0))
output.shape

torch.Size([5, 3, 20])

In [10]:
import torch
import torch.nn as nn
import torch_geometric.nn as gnn
import torch.nn.functional as F

In [11]:
adj_matrix = torch.tensor([
    [0, 1, 0, 0, 0, 0, 0, 0, 0],
    [1, 0, 1, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 1, 1],
    [0, 0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 0, 0],
], dtype=torch.long)


In [68]:
# batch, time, nodes, features
batch, time, nodes, features = 2, 10, 9, 5
outwin, outclasses = 8, 7
input = torch.randn(batch, time, nodes, features)
# create edge_index tensor of size 2 x num_edges
edge_index = adj_matrix.nonzero().t()
print(edge_index.shape)

conv1 = gnn.GCNConv(features, features)
# gatconv = gnn.GATConv(5, 5, 2, concat=False)
# chebconv = gnn.ChebConv(5, 5, 3)
lstmlayer = nn.LSTM(nodes*features, 16, batch_first=True)
lin1 = nn.Linear(16, outclasses)

torch.Size([2, 16])


In [69]:
# input.dim()
input = input.view(-1, features)
x = conv1(input, edge_index)  # B*T*N, F
x.shape

torch.Size([180, 5])

In [70]:
x = x.view(batch, time, -1)
x, _ = lstmlayer(x)
x.shape

torch.Size([2, 10, 16])

In [71]:
x = x[:, -outwin:, :]
x.shape

torch.Size([2, 8, 16])

In [72]:
x = lin1(x)
x.shape

torch.Size([2, 8, 7])

In [73]:
x = F.softmax(x, dim=-1)
x.shape

torch.Size([2, 8, 7])

In [74]:
x = x.view(-1, outclasses)
x.shape

torch.Size([16, 7])

In [75]:
# create y of size batch x time with random integers between 0 and 6
y = torch.randint(0, 7, (batch, outwin))
y = y.view(-1)
y.shape

torch.Size([16])

In [76]:
criteria = nn.CrossEntropyLoss()
criteria(x, y)

tensor(1.9412, grad_fn=<NllLossBackward0>)

In [148]:
input = torch.randn(2, 5, 9, 5)
input

tensor([[[[-1.6138e+00,  6.3239e-01, -2.2832e+00,  1.5254e+00,  6.1059e-01],
          [ 1.0362e+00,  1.2422e+00, -3.9532e-01, -5.8745e-01,  1.4121e+00],
          [-8.9898e-01, -3.8150e-01,  1.6909e+00,  7.6081e-01,  4.5921e-02],
          [ 2.4033e-01,  2.1148e-01, -9.5684e-02,  6.2390e-01,  4.0387e-01],
          [ 4.5510e-01, -1.3155e+00, -4.1844e-01,  1.1429e+00,  3.6841e-01],
          [ 2.1626e+00,  2.9099e-02,  2.0649e-02, -1.4189e+00,  1.4562e+00],
          [ 3.2000e-01,  1.6950e+00, -9.2032e-01,  1.4454e-01, -1.0241e+00],
          [ 6.3031e-01,  4.0703e-01,  4.0836e-01, -6.1746e-01,  2.1072e+00],
          [-3.4915e-01, -7.9907e-01,  1.9772e+00, -2.0166e-01,  1.5471e+00]],

         [[ 1.4314e-01,  1.3646e+00,  1.8309e-01, -9.2163e-01,  1.0790e-01],
          [-3.1746e-01,  1.8292e+00,  1.2066e+00,  4.7163e-01,  7.2576e-01],
          [-1.0275e-01, -3.2090e-01,  9.7740e-01,  6.8282e-01,  1.4440e+00],
          [-9.2377e-01,  1.0290e+00, -3.9652e-01, -5.6842e-01,  1.0524e+00

In [150]:
input = input.view(-1, 5)
input

tensor([[-1.6138e+00,  6.3239e-01, -2.2832e+00,  1.5254e+00,  6.1059e-01],
        [ 1.0362e+00,  1.2422e+00, -3.9532e-01, -5.8745e-01,  1.4121e+00],
        [-8.9898e-01, -3.8150e-01,  1.6909e+00,  7.6081e-01,  4.5921e-02],
        [ 2.4033e-01,  2.1148e-01, -9.5684e-02,  6.2390e-01,  4.0387e-01],
        [ 4.5510e-01, -1.3155e+00, -4.1844e-01,  1.1429e+00,  3.6841e-01],
        [ 2.1626e+00,  2.9099e-02,  2.0649e-02, -1.4189e+00,  1.4562e+00],
        [ 3.2000e-01,  1.6950e+00, -9.2032e-01,  1.4454e-01, -1.0241e+00],
        [ 6.3031e-01,  4.0703e-01,  4.0836e-01, -6.1746e-01,  2.1072e+00],
        [-3.4915e-01, -7.9907e-01,  1.9772e+00, -2.0166e-01,  1.5471e+00],
        [ 1.4314e-01,  1.3646e+00,  1.8309e-01, -9.2163e-01,  1.0790e-01],
        [-3.1746e-01,  1.8292e+00,  1.2066e+00,  4.7163e-01,  7.2576e-01],
        [-1.0275e-01, -3.2090e-01,  9.7740e-01,  6.8282e-01,  1.4440e+00],
        [-9.2377e-01,  1.0290e+00, -3.9652e-01, -5.6842e-01,  1.0524e+00],
        [-1.0799e+00,  1.

In [152]:
input = input.view(2, 5, -1)
input

tensor([[[-1.6138e+00,  6.3239e-01, -2.2832e+00,  1.5254e+00,  6.1059e-01,
           1.0362e+00,  1.2422e+00, -3.9532e-01, -5.8745e-01,  1.4121e+00,
          -8.9898e-01, -3.8150e-01,  1.6909e+00,  7.6081e-01,  4.5921e-02,
           2.4033e-01,  2.1148e-01, -9.5684e-02,  6.2390e-01,  4.0387e-01,
           4.5510e-01, -1.3155e+00, -4.1844e-01,  1.1429e+00,  3.6841e-01,
           2.1626e+00,  2.9099e-02,  2.0649e-02, -1.4189e+00,  1.4562e+00,
           3.2000e-01,  1.6950e+00, -9.2032e-01,  1.4454e-01, -1.0241e+00,
           6.3031e-01,  4.0703e-01,  4.0836e-01, -6.1746e-01,  2.1072e+00,
          -3.4915e-01, -7.9907e-01,  1.9772e+00, -2.0166e-01,  1.5471e+00],
         [ 1.4314e-01,  1.3646e+00,  1.8309e-01, -9.2163e-01,  1.0790e-01,
          -3.1746e-01,  1.8292e+00,  1.2066e+00,  4.7163e-01,  7.2576e-01,
          -1.0275e-01, -3.2090e-01,  9.7740e-01,  6.8282e-01,  1.4440e+00,
          -9.2377e-01,  1.0290e+00, -3.9652e-01, -5.6842e-01,  1.0524e+00,
          -1.0799e+00,  

In [77]:
output = torch.randn(20, 7)
output

tensor([[-1.2988, -2.0953, -1.9439, -0.0422,  0.1306,  0.3742, -0.0788],
        [-0.2944,  2.9790, -0.0356, -1.0226,  1.6753,  1.1189,  1.1823],
        [-0.0696, -0.5503, -0.2964, -0.8635,  2.1796, -0.2591, -0.7912],
        [-1.4386, -0.2678, -0.7445, -0.8222,  1.4264,  2.8123, -0.5126],
        [ 0.8333,  0.2021,  1.2158, -1.0023,  0.0768, -0.1135,  0.5591],
        [-0.4627, -0.7341,  1.5392, -0.7884,  0.6950,  0.1831, -0.8909],
        [ 0.1589,  0.0059, -1.7278,  1.2986,  1.2353,  0.4061,  0.3413],
        [-0.6234, -0.2396,  0.2881, -1.2215,  0.7187,  0.0494, -1.1359],
        [-1.3286,  1.3639,  0.2342, -1.0540,  0.9567, -1.8382,  0.0535],
        [-0.9343, -1.8863, -1.5991, -1.9281, -0.0214,  0.7243, -0.9416],
        [-0.1981,  0.1384,  1.1618, -0.1543, -0.2215, -0.0521, -1.4819],
        [-0.4426,  0.3478,  0.3288,  0.1260,  1.2477,  0.0631, -0.5665],
        [ 1.2982, -0.6365, -0.4069, -0.8846,  0.4556,  0.5436, -2.0896],
        [-1.4886,  0.2036, -0.1400, -2.4167, -0.550

In [78]:
pred = torch.argmax(output, dim=-1)
pred

tensor([5, 1, 4, 5, 2, 2, 3, 4, 1, 5, 2, 4, 0, 6, 5, 1, 0, 2, 5, 2])

In [79]:
_, predicted = torch.max(output, 1)
predicted

tensor([5, 1, 4, 5, 2, 2, 3, 4, 1, 5, 2, 4, 0, 6, 5, 1, 0, 2, 5, 2])

In [91]:
from sklearn.metrics import confusion_matrix
import numpy as np

time1 = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9])
time2 = np.array([1, 2, 4, 4, 5, 6, 7, 8, 9])
y_true = [time1, time1, time1, time1, time1, time1]
y_pred = [time2, time2, time2, time2, time2, time2]
# confusion_matrix(y_true, y_pred)
# y_true = np.array(y_true).reshape(-1)
# y_pred = np.array(y_pred).reshape(-1)
y_true = np.concatenate(y_true).ravel()
y_true

array([1, 2, 3, 4, 5, 6, 7, 8, 9, 1, 2, 3, 4, 5, 6, 7, 8, 9, 1, 2, 3, 4,
       5, 6, 7, 8, 9, 1, 2, 3, 4, 5, 6, 7, 8, 9, 1, 2, 3, 4, 5, 6, 7, 8,
       9, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [150]:
# array of np.inf of size 3
bestval = float('inf')
top3 = []
val = [10, 9, 8, 7, 6, 5, 4, 3, 2, 1]
for i in val:
    if i < bestval:
        bestval = i
        top3.append(i)
        top3.sort()
        top3 = top3[:3]
        for idx, j in enumerate(top3):
            print(f'model{idx+1}, {j}')
    print(f'-----------')


model1, 10
-----------
model1, 9
model2, 10
-----------
model1, 8
model2, 9
model3, 10
-----------
model1, 7
model2, 8
model3, 9
-----------
model1, 6
model2, 7
model3, 8
-----------
model1, 5
model2, 6
model3, 7
-----------
model1, 4
model2, 5
model3, 6
-----------
model1, 3
model2, 4
model3, 5
-----------
model1, 2
model2, 3
model3, 4
-----------
model1, 1
model2, 2
model3, 3
-----------


In [157]:
model_path = "runs/STGCNLSTM/model-top"
model_path.rsplit('/', 1)[0]

'runs/STGCNLSTM'